In [1]:
!git clone https://github.com/yuliya1324/Aspect_Sent_project.git

Cloning into 'Aspect_Sent_project'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 47 (delta 17), reused 33 (delta 10), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.3 MB/s 
     |████████████████████████████████| 7.6 MB 56.9 MB/s 
     |████████████████████████████████| 182 kB 64.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 26.0 MB/s 
     |████████████████████████████████| 174 kB 41.2 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 173 kB 66.0 MB/s 
     |████████████████████████████████| 168 kB 73.4 MB/s 
     |████████████████████████████████| 168 kB 82.3 MB/s 
     |████████████████████████████████| 166 kB 76.5 MB/s 
     |██████

In [1]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: broody_one. Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW, get_linear_schedule_with_warmup, AutoModel
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
from torch import nn, optim
from sklearn.metrics import f1_score, classification_report, accuracy_score

In [3]:
import torch.nn as nn

In [4]:
from transformers import (
    AutoTokenizer, 
    RobertaTokenizerFast, 
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer)

In [5]:
import random

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [6]:
RANDOM_SEED = 42
set_random_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
     

In [7]:
df_tr = pd.read_csv('/content/Aspect_Sent_project/data/aspects_train.csv', index_col=0)
df_va = pd.read_csv('/content/Aspect_Sent_project/data/aspects_val.csv', index_col=0)

In [8]:
df_train = df_tr.groupby("idx").agg(list)
df_val = df_va.groupby("idx").agg(list)

In [9]:
MODEL_NAME = 'sberbank-ai/ruRoberta-large'
MAX_LEN = 512
BATCH_SIZE = 4

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)

In [11]:
class JointDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.cat_to_int = {
            "O": 0, 
            "B-Food": 1, 
            "I-Food": 2, 
            "B-Interior": 3,
            "I-Interior": 4, 
            "B-Price": 5,
            "I-Price": 6, 
            "B-Whole": 7, 
            "I-Whole": 8, 
            "B-Service": 9, 
            "I-Service": 10,
            }
        self.sent_to_int = {"positive": 0, "negative": 1, "neutral": 2, "both": 3}
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset["tokens"])

    def __getitem__(self, idx):
        tokenized_inputs = self.tokenizer(self.dataset["tokens"].iloc[idx], truncation=True, is_split_into_words=True, max_length=512, padding='max_length', return_tensors='pt')
        cat_label = self.dataset["class"].iloc[idx]
        sent_label = self.dataset['sentiment'].iloc[idx]
        word_ids = tokenized_inputs.word_ids()
        previous_word_idx = None
        cat_label_ids = []
        sent_label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                cat_label_ids.append(-100)
                sent_label_ids.append(-100)
            else:
                if cat_label[word_idx] == 'O':
                    sent_label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    sent_label_ids.append(self.sent_to_int[sent_label[word_idx]])
                else:
                    sent_label_ids.append(self.sent_to_int[sent_label[word_idx]])
                if word_idx != previous_word_idx:
                    cat_label_ids.append(self.cat_to_int[cat_label[word_idx]])
                else:
                    cat_label_ids.append(self.cat_to_int[cat_label[word_idx]])
                
            previous_word_idx = word_idx
        tokenized_inputs["input_ids"] = tokenized_inputs["input_ids"].flatten()
        tokenized_inputs["attention_mask"] = tokenized_inputs["attention_mask"].flatten()
        tokenized_inputs["sent_labels"] = torch.Tensor(sent_label_ids)
        tokenized_inputs["cat_labels"] = torch.Tensor(cat_label_ids)
        return tokenized_inputs

In [12]:
class JointClassifier(nn.Module):
  def __init__(self, n_cat, n_sent):
        super(JointClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(MODEL_NAME, add_pooling_layer=False).to(device)
        self.drop = nn.Dropout(p=0.5)
        self.Cat = nn.Linear(self.bert.config.hidden_size, n_cat).to(device)
        self.Sent = nn.Linear(self.bert.config.hidden_size + n_cat, n_sent).to(device)
  
  def forward(self, batch):
        outputs = self.bert(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device)
        )
        cats = self.Cat(self.drop(outputs[0]))
        outputs = self.Sent(torch.cat((outputs[0], cats), 2))
        return {
            "cats": cats,
            "sents": outputs
            }


In [13]:
cur_model = JointClassifier(n_cat=11, n_sent=4)

Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
tr_ds = JointDataset(df_train, tokenizer)
va_ds = JointDataset(df_val, tokenizer)

In [15]:
tr_dl = DataLoader(tr_ds, batch_size=BATCH_SIZE)
va_dl = DataLoader(va_ds, batch_size=BATCH_SIZE)

In [16]:
for name,param in cur_model.named_parameters():
    if not re.search("Sent|Cat|23|22|21|20", name):
        param.requires_grad = False

In [17]:
EPOCHS = 10

optimizer = AdamW(filter(lambda p: p.requires_grad, cur_model.parameters()), lr=1e-5, correct_bias=False, weight_decay=0.06)
total_steps = len(tr_dl) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
def count_loss(batch, pred):
    cat_labs =  batch["cat_labels"].type(torch.LongTensor)
    sent_labs  = batch["sent_labels"].type(torch.LongTensor)
    loss1 = loss_fn(pred["cats"].view(-1, pred["cats"].size(2)), cat_labs.view(-1).to(device))
    loss2 = loss_fn(pred["sents"].view(-1, pred["sents"].size(2)), sent_labs.view(-1).to(device))
    return sum([loss1, loss2])

In [24]:
def compute_metrics(preds, labs):
    res = {}
    for key in preds.keys():
        predictions = np.argmax(preds[key].to('cpu').detach().numpy(), axis=2).reshape((-1,))
        labels = labs[key[:-1]+ '_labels'].reshape((-1,))

        true_predictions = [p for (p, l) in zip(predictions, labels) if l != -100]
        true_labels = [l for (p, l) in zip(predictions, labels) if l != -100]

        accuracy = accuracy_score(true_labels, true_predictions)
        res[key+"accuracy"] = accuracy
    return res

In [22]:
def train_epoch(
  model, 
  data_loader, 
  optimizer, 
  device, 
  scheduler, 
):
  model = model.train()
  
  for batch in data_loader:
    pred = model(batch)
    loss = count_loss(batch, pred) 
    score = compute_metrics(pred, batch)
    score["train/loss"] = loss.item()
    wandb.log(score)
    
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

def eval_model(model, data_loader, device):
    model = model.eval()

    losses = []
    true = {
            "cat_labels": torch.tensor([]), 
            "sent_labels": torch.tensor([]),  
            }
    predictions = {
            "cats": torch.tensor([]), 
            "sents": torch.tensor([]), 
            }
    with torch.no_grad():
        for batch in data_loader:
            pred = model(batch)
            loss = count_loss(batch, pred)
            losses.append(loss.item())

            for k in predictions:
                true[k[:-1]+'_labels'] = torch.cat((true[k[:-1]+'_labels'], batch[k[:-1]+'_labels'].cpu()))
                predictions[k] = torch.cat((predictions[k], pred[k].cpu()))

    score = compute_metrics(predictions, true)
    score["eval/loss"] = np.mean(losses)
    wandb.log(score)

In [25]:
wandb.init(
    project="Sent_aspect",
    name=f"experiment_{2}",
    )

for epoch in range(EPOCHS):
    train_epoch(
        cur_model, 
        tr_dl, 
        optimizer, 
        device, 
        scheduler, 
        )
    eval_model(cur_model, va_dl, device)
wandb.finish()

catsaccuracy,▆█▆▄▇▅▁█
sentsaccuracy,█▆▆▅▁█▇▄
train/loss,▁▃▃▄█▂▃▂
catsaccuracy,0.86496
sentsaccuracy,0.50806
train/loss,1.60658


catsaccuracy,▅▃▄▄▇▁▇▄▆▆▆▆▆▆▆▅█▅▆▅▇▄▅▅▇▆▅▆▆█▇▆█▅▅▅█▄▆▆
eval/loss,█▆▆▄▂▂▁▁▁▁
sentsaccuracy,▃▅▆▁▆▆▃▇▄█▄▆▅▆▆▄▃▆▆▃▇▆▆▇▄█▅▆▆▇▇▆▄▅▇▅▇▇▇█
train/loss,▅▅▄█▃▅▆▃▂▄▅▃▄▅▂▃▄▁▅▃▂▃▃▃▂▂▂▄▂▄▄▂▃▄▂▄▂▂▂▂
catsaccuracy,0.90684
eval/loss,0.90148
sentsaccuracy,0.77607
train/loss,0.74661


In [26]:
torch.save(cur_model, "model_joint.pt")

In [28]:
!git clone https://github.com/pytorch/serve.git

Cloning into 'serve'...
remote: Enumerating objects: 34841, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 34841 (delta 63), reused 109 (delta 53), pack-reused 34698
Receiving objects: 100% (34841/34841), 65.78 MiB | 7.42 MiB/s, done.
Resolving deltas: 100% (20964/20964), done.


In [39]:
!torch-model-archiver --model-name cur_model --version 1.0 --serialized-file model_joint.pt --handler text_classifier --force

WARNING - Overwriting /content/cur_model.mar ...


In [30]:
!pip install torch-model-archiver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
torch-model-archiver

NameError: ignored